<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [6]:
import pandas as pd
import urllib.parse
import urllib.request
import xml.etree.ElementTree as ET
import io
from bs4 import BeautifulSoup
import time
import sys
import multiprocessing as mp

In [1]:
def scrape_all_info2(idlist):
    fieldlist = ['language', 'itemnumber', 'itemname', 'price', 'productsize', 'isdiscontinued', 'islimited',
                 'am', 'ambonusmiles', 'amexpiration', 'kosher', 
                 'lto', 'ltoexpiration', 'ltoregularprice', 'ltosavings', 
                 'releasedate', 'upcnumber', 'sccnumber', 'vao', 'vaodescription', 'vaoexpiration', 
                 'vqa', 'productquantity', 'producer', 'producingsubregion', 'producingregion', 'producingcountry', 
                 'alcoholpercentage', 'sugarcontent', 'tastingnotes', 'pairings', 
                 'itemdescription', 'liquortype', 'categoryname', 'stocktype', 'subcategoryname',
                 'sellingpackage', 'sweetnessdescriptor', 'winestyle', 'winevarietal', 
                 'stylebody', 'stylebody_fr', 'styleflavour', 'styleflavour_fr']

    productinfo = pd.DataFrame(columns=fieldlist)
    
    #timer/counter
    start_time = int(time.time())
    count = 0
    total = len(idlist)
    
    pool = mp.Pool(mp.cpu_count())
    for id in idlist:
        pool.apply_async(parse_item_xml, args=(str(id),fieldlist), callback =collect_result)
        #print(id)
        # Pull xml from lcbo api
        #xmltest = pullXML(str(id))
        # Extract dict of values
        #newitem = parse_item_xml(xmltest, fieldlist)

        # Add to our dataframe
        #productinfo = productinfo.append(newitem,ignore_index=True)
    pool.close()
    pool.join()
    #print(results)
    return results
        #counter/timer
        #end_time = int(time.time())
        #min_spent = round(((end_time-start_time)/60),2)
        #remaining_time = round((min_spent/max(count,1))*max((total-count),1),2)

        #count_str = 'extracted ' + str(count) + '/' + str(total) + \
        #        ' products in total ' + str(min_spent) + ' minutes' + \
        #        ' estimated time remaining is ' + str(remaining_time) + ' minutes'
        #if count % 100 == 0:
        #    sys.stdout.write("\r{}".format(count_str))
        #count += 1
        
        # sleep so we don't get blacklisted
        #time.sleep(0.5)
        
    #save to parquet, also return the pandas dataframe
    #productinfo.to_parquet("productinfo2.parquet")
    #return productinfo

In [2]:
def collect_result(result):
    #print('hi')
    global results
    #print('------')
    #print(result)
    results = results.append(result,ignore_index = True)
    #print(results)

In [3]:
def parse_item_xml(id, fieldlist):
    #print(id)
    #print(fieldlist)
    xmlstring = pullXML(id)
    #print(xmlstring)
    soup = BeautifulSoup(xmlstring)
    product = soup.find('products')
    
    newitem = {}
    #print(fieldlist)
    for field in fieldlist:
        value = product.find(field)
        if value is not None:
            newitem[field] = value.text
    #print(newitem)
    return newitem

In [4]:
def pullXML(ProductID):
    url = 'http://www.foodanddrink.ca/lcbo-webapp/productdetail.do?itemNumber=' + ProductID
    #print(url)
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    values = {'language' : 'en' }
    headers = { 'User-Agent' : user_agent }
    data = urllib.parse.urlencode(values)
    data = data.encode('ascii')
    req = urllib.request.Request(url, data, headers)
    #print('hi')
    try:
        with urllib.request.urlopen(req) as response:
           content = response.read()
        #print(content)
        return content
        #return BeautifulSoup(content, 'xml')
    except :
        #print(response)
        raise response
        

In [ ]:

fieldlist = ['language', 'itemnumber', 'itemname', 'price', 'productsize', 'isdiscontinued', 'islimited',
                 'am', 'ambonusmiles', 'amexpiration', 'kosher', 
                 'lto', 'ltoexpiration', 'ltoregularprice', 'ltosavings', 
                 'releasedate', 'upcnumber', 'sccnumber', 'vao', 'vaodescription', 'vaoexpiration', 
                 'vqa', 'productquantity', 'producer', 'producingsubregion', 'producingregion', 'producingcountry', 
                 'alcoholpercentage', 'sugarcontent', 'tastingnotes', 'pairings', 
                 'itemdescription', 'liquortype', 'categoryname', 'stocktype', 'subcategoryname',
                 'sellingpackage', 'sweetnessdescriptor', 'winestyle', 'winevarietal', 
                 'stylebody', 'stylebody_fr', 'styleflavour', 'styleflavour_fr']

results = pd.DataFrame(columns=fieldlist)

for i in range(14,110):
    maxrange = i * 10000
    minrange = (i-1) * 10000 + 1
    print('--')
    print(str(minrange) + ' to ' + str(maxrange))
    product_info_current = scrape_all_info2(range(minrange,maxrange)).dropna()
    product_info_current.to_parquet('data/product_info'+str(maxrange)+'.parquet', skipna = True)
    

--
130001 to 140000
--
140001 to 150000


/opt/tljh/user/lib/python3.6/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)


--
150001 to 160000
--
160001 to 170000
--
170001 to 180000
--
180001 to 190000
--
190001 to 200000
--
200001 to 210000
--
210001 to 220000
--
220001 to 230000
--
230001 to 240000
--
240001 to 250000
--
250001 to 260000
--
260001 to 270000
--
270001 to 280000
--
280001 to 290000
